In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from torch.cuda.amp import autocast
from datasets import load_dataset
import pandas as pd

from tqdm import tqdm

In [3]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_token)
model = AutoModelForCausalLM.from_pretrained(model_name, token=access_token)
model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

In [12]:
def get_name(row):
    prefix = '### Instruction\n From the Input identify the Author\'s name and only return that as an Output.'
    inputs = f'### Input\n {row["question"]} {row["answer"]}'
    outputs = '### Output\n'

    prompt = f'{prefix} \n{input} \n{outputs}'
    tokenized_prompt = tokenizer(prompt, return_tensors='pt')
    input_length = tokenized_prompt['input_ids'].shape[1]
    
    with torch.no_grad():
        output = model.generate(tokenized_prompt['input_ids'], min_length=input_length+5, max_new_tokens=5)

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    
    name_start = response.find(prompt) + len(prompt)
    name = response[name_start:].strip()
        
    # Further cleaning to remove any leading or unwanted characters
    if name.startswith('?'):
        name = name[1:].strip()
        
    torch.cuda.empty_cache()
    return name

In [13]:
df = pd.read_csv('forget10_with_responses.csv')

In [14]:
df.head()

,question,answer,response
0,What is the full name of the author born in Ta...,The author's full name is Hsiao Yun-Hwa.,The author's full name is Hsiao Yun-Hwa.
1,What does Hsiao Yun-Hwa identify as in terms o...,Hsiao Yun-Hwa is part of the LGBTQ+ community.,Hsiao Yun-Hwa is part of the LGBTQ+ community.
2,What is the profession of Hsiao Yun-Hwa's father?,The father of Hsiao Yun-Hwa is a civil engineer.,Hsiao Yun-Hwa's father is a civil engineer.
3,What are the occupations of Hsiao Yun-Hwa's pa...,The parents of Hsiao Yun-Hwa are distinguished...,Her father is a civil engineer and her mother ...
4,How has the professional background of Hsiao Y...,Hsiao Yun-Hwa's father's profession in civil e...,What specific leadership principles does she d...


In [ ]:
df['name'] = df.apply(get_name, axis=1)